# Beginning state:
- For preliminaries, see "Initial Data Gathering" notebook, with the caveat that I did NOT upload the associated data, so you won't be able to re-run the notebook. As downloaded from the multiple Kaggle datasets there were 4+ GB of data and 200K+ individual files.
- For this notebook, I have combed through those initial datasets to collect data on as many of the 2020 Fortune 100 companies as I could match by ticker/symbol, and produced CSV files of those initial reduced datasets that I was able to upload, so you should be able to re-run this notebook if you have access to my "RawData" folder.

## Goals and notes:
- Concentrating, for now, on balance-sheet data vs price data.
- Goal is to look for correlations between balance-sheet information, price data, and possibly some calculated fields like P/E ratio, market capitalization, etc.
    - I intend to limit price data to the area around the date of each filing, assuming that price movements between the day before/day of/day after the filing are the most likely to reflect the influence of the new information.
    - This means I may need to be pretty careful about getting the "right" release date.
- I have two different datasets for each of market and financial data. Need to make some decisions about whether I collate data or pick one dataset and stick with that.
    - I am suspicious of one of the market datasets; price data seemed artificially low and there is a comment on Kaggle that the creator has "adjusted" all of the price data to account for dividends and splits. Not sure if this will suit my current purpose.
- I suspect most of the work is going to be in the Finnhub data (the bigger financials dataset). Files are currently organized by rows in which each row is a "topic" within a filing, so it's hard to see which topics appear repeatedly and which will turn out to be sparse across many companies and many filings.
    - Beware the sunk costs... I really don't want to toss out this data, I spent a lot of work figuring out how to get it into its current state. But, the other financials dataset is already columnar and will be simpler to work with.
    - On the other hand, that means I don't know how the creator of the simpler dataset did the simplifying.
- My ideal is to produce a dataframe that contains something like:
    - Filing date
    - Filing metadata (quarter/FY, anything else that seems important)
    - Company ticker
    - <balance sheet categories, possibly the least-sparse of them>
    - Close price date - 1
    - Close price date + 0
    - Close price date + 1
    - Stock data like earnings, dividends, and shares outstanding
    

## 1. Start with the simple
- I'm going to pick the "historical financials" aka the simpler SEC dataset, and the "historical prices" market dataset aka the one I'm less suspicious of its price data.

In [1]:
import pandas as pd
import os
from pandasql import sqldf #you may need to install pandasql, didn't come with my Anaconda distro

pysqldf = lambda q: sqldf(q, globals())
data_dir = os.path.join(os.getcwd(), 'RawData')

In [2]:
hf_df = pd.read_csv(os.path.join(data_dir, 'HF-financials-tickers.csv'), parse_dates=['filing_date'], index_col=0)
print(hf_df.head())

    commonstocksharesissued  assetscurrent  accountspayablecurrent  \
11                   1000.0   1.789100e+10            1.625000e+09   
12                   1000.0   1.838000e+10            1.580000e+09   
13                   1000.0   1.705600e+10            1.546000e+09   
14                   1000.0   1.705600e+10            1.546000e+09   
15                   1000.0   1.705600e+10            1.546000e+09   

    commonstockvalue  liabilities  liabilitiesandstockholdersequity  \
11               NaN          NaN                      6.001200e+10   
12               NaN          NaN                      6.079300e+10   
13               NaN          NaN                      5.809200e+10   
14               NaN          NaN                      5.809200e+10   
15               NaN          NaN                      5.809200e+10   

    stockholdersequity  earningspersharebasic  netincomeloss  profitloss  ...  \
11       -7.800000e+08                   3.92   1.282000e+09         Na

In [3]:
print(hf_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2743 entries, 11 to 100690
Data columns (total 44 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   commonstocksharesissued           2471 non-null   float64       
 1   assetscurrent                     2375 non-null   float64       
 2   accountspayablecurrent            2083 non-null   float64       
 3   commonstockvalue                  2160 non-null   float64       
 4   liabilities                       1751 non-null   float64       
 5   liabilitiesandstockholdersequity  2743 non-null   float64       
 6   stockholdersequity                2503 non-null   float64       
 7   earningspersharebasic             2739 non-null   float64       
 8   netincomeloss                     2639 non-null   float64       
 9   profitloss                        1906 non-null   float64       
 10  costofgoodssold                   1168 non-nu

Yup, this data is pretty sparse. I see a few columns off the bat I should drop because they contain no non-null values. 

In [4]:
hf_df2 = hf_df.dropna(axis='columns', how='all').reset_index()
hf_df2.rename({'stock':'ticker', 'filing_date':'date'}, axis=1, inplace=True) # need to normalize column names
print(hf_df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2743 entries, 0 to 2742
Data columns (total 41 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   index                             2743 non-null   int64         
 1   commonstocksharesissued           2471 non-null   float64       
 2   assetscurrent                     2375 non-null   float64       
 3   accountspayablecurrent            2083 non-null   float64       
 4   commonstockvalue                  2160 non-null   float64       
 5   liabilities                       1751 non-null   float64       
 6   liabilitiesandstockholdersequity  2743 non-null   float64       
 7   stockholdersequity                2503 non-null   float64       
 8   earningspersharebasic             2739 non-null   float64       
 9   netincomeloss                     2639 non-null   float64       
 10  profitloss                        1906 non-null 

While I think about "how sparse is too sparse" for some of these columns, will try pulling in share price and earnings data.

In [5]:
pr_df = pd.read_csv(os.path.join(data_dir, 'HP-market-ticker-prices.csv'), index_col=0, parse_dates=['date'])
print(pr_df.head())

  symbol       date   open   high    low  close  close_adjusted     volume  \
0   MSFT 2016-05-16  50.80  51.96  50.75  51.83         49.7013   20032017   
1   MSFT 2002-01-16  68.85  69.84  67.85  67.87         22.5902   30977700   
2   MSFT 2001-09-18  53.41  55.00  53.17  54.32         18.0802   41591300   
3   MSFT 2007-10-26  36.01  36.03  34.56  35.03         27.2232  288121200   
4   MSFT 2014-06-27  41.61  42.29  41.51  42.25         38.6773   74640000   

   split_coefficient  
0                1.0  
1                1.0  
2                1.0  
3                1.0  
4                1.0  


In [6]:
print(pr_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 437565 entries, 0 to 23526281
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   symbol             437565 non-null  object        
 1   date               437565 non-null  datetime64[ns]
 2   open               437565 non-null  float64       
 3   high               437565 non-null  float64       
 4   low                437565 non-null  float64       
 5   close              437565 non-null  float64       
 6   close_adjusted     437565 non-null  float64       
 7   volume             437565 non-null  int64         
 8   split_coefficient  437565 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 33.4+ MB
None


If I multi-index these by ticker and date, will that make it easier to combine them?

In [7]:
pr_df.rename({'symbol':'ticker'}, axis=1, inplace=True) # need to normalize what the ticker/symbol column name is
pr_df2 = pr_df.set_index(['ticker', 'date']).sort_index()
print(pr_df2.info())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 437565 entries, ('AAL', Timestamp('2005-09-27 00:00:00')) to ('XOM', Timestamp('2020-11-11 00:00:00'))
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   open               437565 non-null  float64
 1   high               437565 non-null  float64
 2   low                437565 non-null  float64
 3   close              437565 non-null  float64
 4   close_adjusted     437565 non-null  float64
 5   volume             437565 non-null  int64  
 6   split_coefficient  437565 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 24.8+ MB
None


In [8]:
print(pr_df2.head())

                    open   high    low  close  close_adjusted    volume  \
ticker date                                                               
AAL    2005-09-27  21.05  21.40  19.10  19.30         18.6645   2576944   
       2005-09-28  19.30  20.53  19.20  20.50         19.8250  15409920   
       2005-09-29  20.40  20.58  20.10  20.21         19.5445   2890617   
       2005-09-30  20.26  21.05  20.18  21.01         20.3182   8373458   
       2005-10-03  20.90  21.75  20.90  21.50         20.7920   2836193   

                   split_coefficient  
ticker date                           
AAL    2005-09-27                1.0  
       2005-09-28                1.0  
       2005-09-29                1.0  
       2005-09-30                1.0  
       2005-10-03                1.0  


In [9]:
hf_df3 = hf_df2.set_index(['ticker', 'date']).sort_index()
print(hf_df3.info())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2743 entries, ('AAL', Timestamp('2010-07-21 00:00:00')) to ('XOM', Timestamp('2019-05-02 00:00:00'))
Data columns (total 39 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   index                             2743 non-null   int64  
 1   commonstocksharesissued           2471 non-null   float64
 2   assetscurrent                     2375 non-null   float64
 3   accountspayablecurrent            2083 non-null   float64
 4   commonstockvalue                  2160 non-null   float64
 5   liabilities                       1751 non-null   float64
 6   liabilitiesandstockholdersequity  2743 non-null   float64
 7   stockholdersequity                2503 non-null   float64
 8   earningspersharebasic             2739 non-null   float64
 9   netincomeloss                     2639 non-null   float64
 10  profitloss                        1906 non-null   float

In [10]:
hf_df3.drop(labels='index', axis=1, inplace=True)
print(hf_df3.info())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2743 entries, ('AAL', Timestamp('2010-07-21 00:00:00')) to ('XOM', Timestamp('2019-05-02 00:00:00'))
Data columns (total 38 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   commonstocksharesissued           2471 non-null   float64
 1   assetscurrent                     2375 non-null   float64
 2   accountspayablecurrent            2083 non-null   float64
 3   commonstockvalue                  2160 non-null   float64
 4   liabilities                       1751 non-null   float64
 5   liabilitiesandstockholdersequity  2743 non-null   float64
 6   stockholdersequity                2503 non-null   float64
 7   earningspersharebasic             2739 non-null   float64
 8   netincomeloss                     2639 non-null   float64
 9   profitloss                        1906 non-null   float64
 10  costofgoodssold                   1168 non-null   float

In [11]:
print(hf_df3.head())

                   commonstocksharesissued  assetscurrent  \
ticker date                                                 
AAL    2010-07-21              339389724.0   7.344000e+09   
       2010-10-20              339389724.0   6.837000e+09   
       2011-02-16              339389724.0   6.838000e+09   
       2011-04-20              341207797.0   8.825000e+09   
       2011-07-20              341207797.0   7.997000e+09   

                   accountspayablecurrent  commonstockvalue  liabilities  \
ticker date                                                                
AAL    2010-07-21            1.305000e+09       339000000.0          NaN   
       2010-10-20            1.220000e+09       339000000.0          NaN   
       2011-02-16            1.156000e+09       339000000.0          NaN   
       2011-04-20            1.267000e+09       339000000.0          NaN   
       2011-07-20            1.291000e+09       341000000.0          NaN   

                   liabilitiesandstockh

### Pull out 3 close prices per filing date

In [12]:
# in pseudocode, I want to:
# for each ticker and filing date in hf_df (currently df3):
#     retrieve the closing price for the same ticker and date from pr_df(currently df2), stored as a new column in hf_df
#     retrieve the closing price for the same ticker and date-1, stored as another new column in hf_df
#     retrieve the closing price for the same ticker and date+1, stored as a third new column in hf_df
# option on storing the 3 columns in their own dataframe on the way to merging into hf_df

# research:
# https://stackoverflow.com/questions/50655370/filtering-the-dataframe-based-on-the-column-value-of-another-dataframe
# https://datascience.stackexchange.com/questions/47562/multiple-filtering-pandas-columns-based-on-values-in-another-column
# https://pandas.pydata.org/docs/user_guide/indexing.html#selection-by-callable
# also looking at merge (inner join could work for same-date, but how to only get the "close" column?)
# indexing with query or where methods?

# I suspect I need to make my "date" fields the same name for indexes to match... done that

test_df = hf_df3.merge(pr_df2, left_index=True, right_index=True, how='left')

In [13]:
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2743 entries, ('AAL', Timestamp('2010-07-21 00:00:00')) to ('XOM', Timestamp('2019-05-02 00:00:00'))
Data columns (total 45 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   commonstocksharesissued           2471 non-null   float64
 1   assetscurrent                     2375 non-null   float64
 2   accountspayablecurrent            2083 non-null   float64
 3   commonstockvalue                  2160 non-null   float64
 4   liabilities                       1751 non-null   float64
 5   liabilitiesandstockholdersequity  2743 non-null   float64
 6   stockholdersequity                2503 non-null   float64
 7   earningspersharebasic             2739 non-null   float64
 8   netincomeloss                     2639 non-null   float64
 9   profitloss                        1906 non-null   float64
 10  costofgoodssold                   1168 non-null   float

This is good for same-date. Do I want to also keep volume info for all 3 dates? I'd like to try...

In [14]:
# Mentor pointed me toward pandasql: https://pypi.org/project/pandasql/
# because I think I know how to query for (date = date-1) in SQL and doesn't seem to be a great way to do it with only Pandas
dateminus_q = """SELECT pr_df2.ticker, pr_df2.date AS date_minus1, hf_df3.date, open AS open_minus1, high AS high_minus1, low AS low_minus1, close AS close_minus1, 
            close_adjusted AS close_adj_minus1, volume AS volume_minus1, split_coefficient AS split_coef_minus1
        FROM pr_df2
        LEFT JOIN hf_df3 ON pr_df2.ticker = hf_df3.ticker
        WHERE DATE(pr_df2.date) = DATE(hf_df3.date, '-1 day');"""
dateminus_df = pysqldf(dateminus_q)
print(dateminus_df.head())
print(test_df.iloc[:6, 38:45]) #checking to see if the price data really is different

  ticker                 date_minus1                        date  open_minus1  \
0    AAL  2010-07-20 00:00:00.000000  2010-07-21 00:00:00.000000         8.50   
1    AAL  2010-10-19 00:00:00.000000  2010-10-20 00:00:00.000000         9.94   
2    AAL  2011-02-15 00:00:00.000000  2011-02-16 00:00:00.000000        10.07   
3    AAL  2011-04-19 00:00:00.000000  2011-04-20 00:00:00.000000         8.33   
4    AAL  2011-07-19 00:00:00.000000  2011-07-20 00:00:00.000000         7.08   

   high_minus1  low_minus1  close_minus1  close_adj_minus1  volume_minus1  \
0         8.95        8.50          8.92            8.6263       18199196   
1        10.24        9.85         10.09            9.7577       21976408   
2        10.14        9.97         10.08            9.7481       10175603   
3         8.53        8.23          8.25            7.9783       13090885   
4         7.08        6.79          7.01            6.7792       26153351   

   split_coef_minus1  
0                1.0  
1   

In [15]:
print(dateminus_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2420 entries, 0 to 2419
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ticker             2420 non-null   object 
 1   date_minus1        2420 non-null   object 
 2   date               2420 non-null   object 
 3   open_minus1        2420 non-null   float64
 4   high_minus1        2420 non-null   float64
 5   low_minus1         2420 non-null   float64
 6   close_minus1       2420 non-null   float64
 7   close_adj_minus1   2420 non-null   float64
 8   volume_minus1      2420 non-null   int64  
 9   split_coef_minus1  2420 non-null   float64
dtypes: float64(6), int64(1), object(3)
memory usage: 189.2+ KB
None


Not quite the same number of rows! Probably places where filing was on a Monday so there is no Sunday price info. Not sure I should spend 3 days trying to figure out how to code around weekends for the sake of ~300 rows. Keep moving.

In [16]:
dateminus_df['date'] = pd.to_datetime(dateminus_df['date'])
dateminus_df['date_minus1'] = pd.to_datetime(dateminus_df['date_minus1'])
dateminus_df2 = dateminus_df.set_index(['ticker', 'date']).sort_index()
print(dateminus_df2.info())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2420 entries, ('AAL', Timestamp('2010-07-21 00:00:00')) to ('XOM', Timestamp('2019-05-02 00:00:00'))
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date_minus1        2420 non-null   datetime64[ns]
 1   open_minus1        2420 non-null   float64       
 2   high_minus1        2420 non-null   float64       
 3   low_minus1         2420 non-null   float64       
 4   close_minus1       2420 non-null   float64       
 5   close_adj_minus1   2420 non-null   float64       
 6   volume_minus1      2420 non-null   int64         
 7   split_coef_minus1  2420 non-null   float64       
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 209.2+ KB
None


In [17]:
test_df2 = test_df.merge(dateminus_df2, left_index=True, right_index=True, how='left', indicator=True)
print(test_df2.info())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2749 entries, ('AAL', Timestamp('2010-07-21 00:00:00')) to ('XOM', Timestamp('2019-05-02 00:00:00'))
Data columns (total 54 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   commonstocksharesissued           2477 non-null   float64       
 1   assetscurrent                     2381 non-null   float64       
 2   accountspayablecurrent            2089 non-null   float64       
 3   commonstockvalue                  2166 non-null   float64       
 4   liabilities                       1757 non-null   float64       
 5   liabilitiesandstockholdersequity  2749 non-null   float64       
 6   stockholdersequity                2503 non-null   float64       
 7   earningspersharebasic             2745 non-null   float64       
 8   netincomeloss                     2645 non-null   float64       
 9   profitloss                        1912 no

Um... what happened to my price info? There used to be more rows.... whoopsie, I need a left join.
Reran with a left join and I've gained 6 rows overall, what?

In [18]:
print(test_df2.iloc[:6, 38:53]) #looking at just the price columns

                    open   high    low  close  close_adjusted      volume  \
ticker date                                                                 
AAL    2010-07-21   9.45   9.81   9.16   9.21          8.9067  55215818.0   
       2010-10-20  10.50  11.05  10.40  10.84         10.4830  53324397.0   
       2011-02-16  10.15  10.21   9.87  10.10          9.7674  12961662.0   
       2011-04-20   8.35   8.45   8.09   8.14          7.8720  13698123.0   
       2011-07-20   7.12   7.14   6.86   6.90          6.6728  14597319.0   
       2011-10-19   6.02   6.07   5.70   5.76          5.5703  19617694.0   

                   split_coefficient date_minus1  open_minus1  high_minus1  \
ticker date                                                                  
AAL    2010-07-21                1.0  2010-07-20         8.50         8.95   
       2010-10-20                1.0  2010-10-19         9.94        10.24   
       2011-02-16                1.0  2011-02-15        10.07        10

In [19]:
# trying to find these 6 extra rows.

'''
# https://stackoverflow.com/questions/28901683/pandas-get-rows-which-are-not-in-other-dataframe

df_all = test_df2.merge(test_df.drop_duplicates(), left_index=True, right_index=True, 
                   how='left', indicator=True)
print(df_all['_merge'].unique())
'''
# this way produces 2767 rows whose indicator column all say "both", which is a gain of 18 rows over 
# the 2749 number I was trying to solve
'''
# https://stackoverflow.com/questions/33349797/compare-pandas-dataframes-and-return-rows-that-are-missing-from-the-first-one/42004293#42004293

df = pd.merge(test_df, test_df2, how='outer', suffixes=('','_y'), indicator=True)
print(df['_merge'].unique())
'''
# this way produces the expected 2749 rows, and again the indicator all are "both", so I still don't know where the 6 
# extra rows in test_df2 are.

# Maybe go back and put indicator in the test_df2 merge command
;

"\n# https://stackoverflow.com/questions/33349797/compare-pandas-dataframes-and-return-rows-that-are-missing-from-the-first-one/42004293#42004293\n\ndf = pd.merge(test_df, test_df2, how='outer', suffixes=('','_y'), indicator=True)\nprint(df['_merge'].unique())\n"

In [20]:
print(test_df2['_merge'].unique())

['both', 'left_only']
Categories (2, object): ['both', 'left_only']


In [22]:
print(test_df2[test_df2['_merge'] == 'left_only'].info())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 323 entries, ('AAL', Timestamp('2019-02-25 00:00:00')) to ('WMT', Timestamp('2018-06-04 00:00:00'))
Data columns (total 54 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   commonstocksharesissued           299 non-null    float64       
 1   assetscurrent                     265 non-null    float64       
 2   accountspayablecurrent            244 non-null    float64       
 3   commonstockvalue                  235 non-null    float64       
 4   liabilities                       191 non-null    float64       
 5   liabilitiesandstockholdersequity  323 non-null    float64       
 6   stockholdersequity                289 non-null    float64       
 7   earningspersharebasic             321 non-null    float64       
 8   netincomeloss                     310 non-null    float64       
 9   profitloss                        252 non-

Okay, so 2420 rows from the 'dateminus' query plus 323 rows in "left only" is 2743. How about the other way?

In [32]:
print(test_df2[test_df2['_merge'] == 'both'].info())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2426 entries, ('AAL', Timestamp('2010-07-21 00:00:00')) to ('XOM', Timestamp('2019-05-02 00:00:00'))
Data columns (total 54 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   commonstocksharesissued           2178 non-null   float64       
 1   assetscurrent                     2116 non-null   float64       
 2   accountspayablecurrent            1845 non-null   float64       
 3   commonstockvalue                  1931 non-null   float64       
 4   liabilities                       1566 non-null   float64       
 5   liabilitiesandstockholdersequity  2426 non-null   float64       
 6   stockholdersequity                2214 non-null   float64       
 7   earningspersharebasic             2424 non-null   float64       
 8   netincomeloss                     2335 non-null   float64       
 9   profitloss                        1660 no

So the 'minus1' data has gained 6 non-null rows during the merge but they're all found on both sides. Whut?

The 'day of' price data has also gained 6 non-null rows.

In [33]:
print(test_df2[test_df2.duplicated(keep=False)])

                   commonstocksharesissued  assetscurrent  \
ticker date                                                 
TECD   2014-02-25               59239085.0   6.145425e+09   
       2014-02-25               59239085.0   6.145425e+09   
       2014-02-25               59239085.0   6.145425e+09   
       2014-02-25               59239085.0   5.584149e+09   
       2014-02-25               59239085.0   5.584149e+09   
       2014-02-25               59239085.0   5.584149e+09   
       2014-02-25               59239085.0   5.449011e+09   
       2014-02-25               59239085.0   5.449011e+09   
       2014-02-25               59239085.0   5.449011e+09   

                   accountspayablecurrent  commonstockvalue   liabilities  \
ticker date                                                                 
TECD   2014-02-25            3.657251e+09           89000.0  4.711173e+09   
       2014-02-25            3.657251e+09           89000.0  4.711173e+09   
       2014-02-25   

In [34]:
print(test_df[test_df.duplicated(keep=False)])

Empty DataFrame
Columns: [commonstocksharesissued, assetscurrent, accountspayablecurrent, commonstockvalue, liabilities, liabilitiesandstockholdersequity, stockholdersequity, earningspersharebasic, netincomeloss, profitloss, costofgoodssold, costsandexpenses, cash, notespayable, preferredstockvalue, depreciation, operatingexpenses, revenues, land, accountsreceivablenet, deferredrevenue, grossprofit, sharesissued, accruedincometaxes, sharesoutstanding, borrowedfunds, inventorygross, commercialpaper, dividends, commonstocknoparvalue, costofservices, debtcurrent, accruedinsurancecurrent, officerscompensation, salariesandwages, interestanddebtexpense, convertibledebt, operatingcycle, open, high, low, close, close_adjusted, volume, split_coefficient]
Index: []

[0 rows x 45 columns]


In [36]:
print(test_df.loc[('TECD', '2014-02-25'), :])

                   commonstocksharesissued  assetscurrent  \
ticker date                                                 
TECD   2014-02-25               59239085.0   6.145425e+09   
       2014-02-25               59239085.0   5.584149e+09   
       2014-02-25               59239085.0   5.449011e+09   

                   accountspayablecurrent  commonstockvalue   liabilities  \
ticker date                                                                 
TECD   2014-02-25            3.657251e+09           89000.0  4.711173e+09   
       2014-02-25            3.657251e+09           89000.0  4.331335e+09   
       2014-02-25            3.657251e+09           89000.0  4.221714e+09   

                   liabilitiesandstockholdersequity  stockholdersequity  \
ticker date                                                               
TECD   2014-02-25                      6.830960e+09                 NaN   
       2014-02-25                      6.830960e+09                 NaN   
       2

Well, there it is... this one ticker has 3 different sets of filing numbers for the same date. Not sure why they survived the merge of day-of prices but then tripled themselves when I pulled in day-before prices.

In [37]:
print(hf_df3.loc[('TECD', '2014-02-25'), :])

                   commonstocksharesissued  assetscurrent  \
ticker date                                                 
TECD   2014-02-25               59239085.0   6.145425e+09   
       2014-02-25               59239085.0   5.584149e+09   
       2014-02-25               59239085.0   5.449011e+09   

                   accountspayablecurrent  commonstockvalue   liabilities  \
ticker date                                                                 
TECD   2014-02-25            3.657251e+09           89000.0  4.711173e+09   
       2014-02-25            3.657251e+09           89000.0  4.331335e+09   
       2014-02-25            3.657251e+09           89000.0  4.221714e+09   

                   liabilitiesandstockholdersequity  stockholdersequity  \
ticker date                                                               
TECD   2014-02-25                      6.830960e+09                 NaN   
       2014-02-25                      6.830960e+09                 NaN   
       2

In [38]:
print(dateminus_df2.loc[('TECD', '2014-02-25'), :])

                  date_minus1  open_minus1  high_minus1  low_minus1  \
ticker date                                                           
TECD   2014-02-25  2014-02-24        58.37        58.92       57.99   
       2014-02-25  2014-02-24        58.37        58.92       57.99   
       2014-02-25  2014-02-24        58.37        58.92       57.99   

                   close_minus1  close_adj_minus1  volume_minus1  \
ticker date                                                        
TECD   2014-02-25         58.25             58.25         226900   
       2014-02-25         58.25             58.25         226900   
       2014-02-25         58.25             58.25         226900   

                   split_coef_minus1  
ticker date                           
TECD   2014-02-25                1.0  
       2014-02-25                1.0  
       2014-02-25                1.0  


Wierd. Anyway, I think that's it, so let's go back and pull in the dateplus data and clean up the duplicates.

In [39]:
test_df3 = test_df2.drop_duplicates()
print(test_df3.info())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2743 entries, ('AAL', Timestamp('2010-07-21 00:00:00')) to ('XOM', Timestamp('2019-05-02 00:00:00'))
Data columns (total 54 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   commonstocksharesissued           2471 non-null   float64       
 1   assetscurrent                     2375 non-null   float64       
 2   accountspayablecurrent            2083 non-null   float64       
 3   commonstockvalue                  2160 non-null   float64       
 4   liabilities                       1751 non-null   float64       
 5   liabilitiesandstockholdersequity  2743 non-null   float64       
 6   stockholdersequity                2503 non-null   float64       
 7   earningspersharebasic             2739 non-null   float64       
 8   netincomeloss                     2639 non-null   float64       
 9   profitloss                        1906 no

In [40]:
dateplus_q = """SELECT pr_df2.ticker, pr_df2.date AS date_plus1, hf_df3.date, open AS open_plus1, high AS high_plus1, 
            low AS low_plus1, close AS close_plus1, close_adjusted AS close_adj_plus1, volume AS volume_plus1, 
            split_coefficient AS split_coef_plus1
        FROM pr_df2
        LEFT JOIN hf_df3 ON pr_df2.ticker = hf_df3.ticker
        WHERE DATE(pr_df2.date) = DATE(hf_df3.date, '+1 day');"""
dateplus_df = pysqldf(dateplus_q)
dateplus_df['date'] = pd.to_datetime(dateplus_df['date'])
dateplus_df['date_plus1'] = pd.to_datetime(dateplus_df['date_plus1'])
dateplus_df2 = dateplus_df.set_index(['ticker', 'date']).sort_index()
print(dateplus_df2.info())
print(dateplus_df2.head())
print(test_df3.iloc[:6, 38:45]) #checking to see if the price data really is different

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2122 entries, ('AAL', Timestamp('2010-07-21 00:00:00')) to ('XOM', Timestamp('2019-05-02 00:00:00'))
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date_plus1        2122 non-null   datetime64[ns]
 1   open_plus1        2122 non-null   float64       
 2   high_plus1        2122 non-null   float64       
 3   low_plus1         2122 non-null   float64       
 4   close_plus1       2122 non-null   float64       
 5   close_adj_plus1   2122 non-null   float64       
 6   volume_plus1      2122 non-null   int64         
 7   split_coef_plus1  2122 non-null   float64       
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 189.1+ KB
None
                  date_plus1  open_plus1  high_plus1  low_plus1  close_plus1  \
ticker date                                                                    
AAL    2010-07-21 2010-07-22        9.45

In [41]:
test_df4 = test_df3.merge(dateplus_df2, left_index=True, right_index=True, how='left')
print(test_df4.info())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2749 entries, ('AAL', Timestamp('2010-07-21 00:00:00')) to ('XOM', Timestamp('2019-05-02 00:00:00'))
Data columns (total 62 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   commonstocksharesissued           2477 non-null   float64       
 1   assetscurrent                     2381 non-null   float64       
 2   accountspayablecurrent            2089 non-null   float64       
 3   commonstockvalue                  2166 non-null   float64       
 4   liabilities                       1757 non-null   float64       
 5   liabilitiesandstockholdersequity  2749 non-null   float64       
 6   stockholdersequity                2503 non-null   float64       
 7   earningspersharebasic             2745 non-null   float64       
 8   netincomeloss                     2645 non-null   float64       
 9   profitloss                        1912 no

In [42]:
print(test_df4[test_df4.duplicated(keep=False)])

                   commonstocksharesissued  assetscurrent  \
ticker date                                                 
TECD   2014-02-25               59239085.0   6.145425e+09   
       2014-02-25               59239085.0   6.145425e+09   
       2014-02-25               59239085.0   6.145425e+09   
       2014-02-25               59239085.0   5.584149e+09   
       2014-02-25               59239085.0   5.584149e+09   
       2014-02-25               59239085.0   5.584149e+09   
       2014-02-25               59239085.0   5.449011e+09   
       2014-02-25               59239085.0   5.449011e+09   
       2014-02-25               59239085.0   5.449011e+09   

                   accountspayablecurrent  commonstockvalue   liabilities  \
ticker date                                                                 
TECD   2014-02-25            3.657251e+09           89000.0  4.711173e+09   
       2014-02-25            3.657251e+09           89000.0  4.711173e+09   
       2014-02-25   

Same thing! Even though I said left merge, it's keeping all the possibilities for 3x3 duplicate indexes between the two frames.

In [43]:
test_df4 = test_df4.drop_duplicates()

In [44]:
print(test_df4.info())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2743 entries, ('AAL', Timestamp('2010-07-21 00:00:00')) to ('XOM', Timestamp('2019-05-02 00:00:00'))
Data columns (total 62 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   commonstocksharesissued           2471 non-null   float64       
 1   assetscurrent                     2375 non-null   float64       
 2   accountspayablecurrent            2083 non-null   float64       
 3   commonstockvalue                  2160 non-null   float64       
 4   liabilities                       1751 non-null   float64       
 5   liabilitiesandstockholdersequity  2743 non-null   float64       
 6   stockholdersequity                2503 non-null   float64       
 7   earningspersharebasic             2739 non-null   float64       
 8   netincomeloss                     2639 non-null   float64       
 9   profitloss                        1906 no

In [45]:
final_df = test_df4.drop(columns='_merge')
print(final_df.info())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2743 entries, ('AAL', Timestamp('2010-07-21 00:00:00')) to ('XOM', Timestamp('2019-05-02 00:00:00'))
Data columns (total 61 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   commonstocksharesissued           2471 non-null   float64       
 1   assetscurrent                     2375 non-null   float64       
 2   accountspayablecurrent            2083 non-null   float64       
 3   commonstockvalue                  2160 non-null   float64       
 4   liabilities                       1751 non-null   float64       
 5   liabilitiesandstockholdersequity  2743 non-null   float64       
 6   stockholdersequity                2503 non-null   float64       
 7   earningspersharebasic             2739 non-null   float64       
 8   netincomeloss                     2639 non-null   float64       
 9   profitloss                        1906 no

In [46]:
print(final_df.head())

                   commonstocksharesissued  assetscurrent  \
ticker date                                                 
AAL    2010-07-21              339389724.0   7.344000e+09   
       2010-10-20              339389724.0   6.837000e+09   
       2011-02-16              339389724.0   6.838000e+09   
       2011-04-20              341207797.0   8.825000e+09   
       2011-07-20              341207797.0   7.997000e+09   

                   accountspayablecurrent  commonstockvalue  liabilities  \
ticker date                                                                
AAL    2010-07-21            1.305000e+09       339000000.0          NaN   
       2010-10-20            1.220000e+09       339000000.0          NaN   
       2011-02-16            1.156000e+09       339000000.0          NaN   
       2011-04-20            1.267000e+09       339000000.0          NaN   
       2011-07-20            1.291000e+09       341000000.0          NaN   

                   liabilitiesandstockh

In [47]:
print(final_df.iloc[:6, 38:61])

                    open   high    low  close  close_adjusted      volume  \
ticker date                                                                 
AAL    2010-07-21   9.45   9.81   9.16   9.21          8.9067  55215818.0   
       2010-10-20  10.50  11.05  10.40  10.84         10.4830  53324397.0   
       2011-02-16  10.15  10.21   9.87  10.10          9.7674  12961662.0   
       2011-04-20   8.35   8.45   8.09   8.14          7.8720  13698123.0   
       2011-07-20   7.12   7.14   6.86   6.90          6.6728  14597319.0   
       2011-10-19   6.02   6.07   5.70   5.76          5.5703  19617694.0   

                   split_coefficient date_minus1  open_minus1  high_minus1  \
ticker date                                                                  
AAL    2010-07-21                1.0  2010-07-20         8.50         8.95   
       2010-10-20                1.0  2010-10-19         9.94        10.24   
       2011-02-16                1.0  2011-02-15        10.07        10

In [48]:
final_df.to_csv('hf-3-day-prices.csv')